<a href="https://colab.research.google.com/github/Aneesh9390/Data-Science-Notes/blob/main/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the dependencies
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import operator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.sparse import csr_matrix

#load libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

##The %matplotlib inline command tells the IPython environment to draw the plots immediately after the current cell
%matplotlib inline
##below cmd is used to set the figure size -> (width, height) in inches
plt.rcParams['figure.figsize'] = (10.0, 8.0)
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading data
df = pd.read_csv('/content/drive/MyDrive/train.csv') # changed path as present in my drive
df = df.sample(n=200000)

In [ ]:
df.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
904071,695687,Lymphatic filariasis - Elephantiasis,NaN,NaN,1,591.000
1589181,1480442,SOFYANA Girl's Synthetic a-line Maxi (SY-Dress...,"[Color Name: Blue & White,20% Cotton 80% Polye...",NaN,2738,984.250
2176487,1477669,MEISTAR French Country Style Wooden Decoration...,[【HIGH-QUALITY SILENT QUARTZ MOVEMENT】Using hi...,Thickness:0.35 inches(about 9mm)<br> Materials...,1574,1600.000
1540584,274477,Les Aventures de Telemaques Sivies Des Aventur...,NaN,NaN,98,744.093
2122598,97645,"Bury Him Deeper: Love, Passion and Other Sordi...",NaN,NaN,12743,600.000


In [ ]:
df['TITLE'] = df['TITLE'].fillna('')
df['BULLET_POINTS'] = df['BULLET_POINTS'].fillna('')
df['DESCRIPTION'] = df['TITLE'].fillna('')

In [ ]:
df['data'] = df['TITLE'] + " " + df['BULLET_POINTS'] + " " + df['DESCRIPTION']

In [ ]:
del df['BULLET_POINTS']
del df['DESCRIPTION']
del df['TITLE']

In [ ]:
df['data']

904071     Lymphatic filariasis - Elephantiasis  Lymphati...
1589181    SOFYANA Girl's Synthetic a-line Maxi (SY-Dress...
2176487    MEISTAR French Country Style Wooden Decoration...
1540584    Les Aventures de Telemaques Sivies Des Aventur...
2122598    Bury Him Deeper: Love, Passion and Other Sordi...
                                 ...                        
182487     ELECTRICAL POWER SYSTEMS ( 7th ED.)  ELECTRICA...
1171634    YGM Acoustic Foams® Turbo Acoustic Panels Set ...
738220     Mother Goose - From Nursery to Literature  Mot...
2172353    KAKASA Ultimate Trust Real Perfect Fit Soft Sl...
457918     RENESMEE® Professional 10Pcs Beauty Makeup Bru...
Name: data, Length: 200000, dtype: object

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

"""def remove_stopwords(text):
    tokens = nltk.word_tokenize(str(text))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

df['filtered_text'] = df['data'].apply(lambda x: remove_stopwords(x))
print(df['filtered_text'])"""

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"def remove_stopwords(text):\n    tokens = nltk.word_tokenize(str(text))\n    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]\n    filtered_text = ' '.join(filtered_tokens)\n    return filtered_text\n\ndf['filtered_text'] = df['data'].apply(lambda x: remove_stopwords(x))\nprint(df['filtered_text'])"

In [ ]:
def remove_html_tags(text):
  return re.sub(r'<.*?>', '', text)

In [ ]:
df['data'] = df['data'].apply(lambda x: remove_html_tags(x))

In [ ]:
df.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,data
904071,695687,1,591.000,Lymphatic filariasis - Elephantiasis Lymphati...
1589181,1480442,2738,984.250,SOFYANA Girl's Synthetic a-line Maxi (SY-Dress...
2176487,1477669,1574,1600.000,MEISTAR French Country Style Wooden Decoration...
1540584,274477,98,744.093,Les Aventures de Telemaques Sivies Des Aventur...
2122598,97645,12743,600.000,"Bury Him Deeper: Love, Passion and Other Sordi..."


In [ ]:
df['data'] = df['data'].str.lower()

In [ ]:
df.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,data
904071,695687,1,591.000,lymphatic filariasis - elephantiasis lymphati...
1589181,1480442,2738,984.250,sofyana girl's synthetic a-line maxi (sy-dress...
2176487,1477669,1574,1600.000,meistar french country style wooden decoration...
1540584,274477,98,744.093,les aventures de telemaques sivies des aventur...
2122598,97645,12743,600.000,"bury him deeper: love, passion and other sordi..."


In [ ]:
def remove_punctuations(text):
  punctuations = string.punctuation

  return text.translate(str.maketrans('', '', punctuations))

In [ ]:
df['data'] = df['data'].apply(lambda x: remove_punctuations(x))

In [ ]:
df.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,data
904071,695687,1,591.000,lymphatic filariasis elephantiasis lymphatic...
1589181,1480442,2738,984.250,sofyana girls synthetic aline maxi sydress1884...
2176487,1477669,1574,1600.000,meistar french country style wooden decoration...
1540584,274477,98,744.093,les aventures de telemaques sivies des aventur...
2122598,97645,12743,600.000,bury him deeper love passion and other sordid ...


In [ ]:
def remove_stopwords(text):
  return ' '.join([word for word in text.split() if word not in stop_words])

In [ ]:
df['data'] = df['data'].apply(lambda x: remove_stopwords(x))

In [ ]:
df.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,data
904071,695687,1,591.000,lymphatic filariasis elephantiasis lymphatic f...
1589181,1480442,2738,984.250,sofyana girls synthetic aline maxi sydress1884...
2176487,1477669,1574,1600.000,meistar french country style wooden decoration...
1540584,274477,98,744.093,les aventures de telemaques sivies des aventur...
2122598,97645,12743,600.000,bury deeper love passion sordid affairs bury d...


In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

In [ ]:
df['data'] = df['data'].apply(lambda x: lemmatize_text(x))

In [ ]:
df.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,data
904071,695687,1,591.000,lymphatic filariasis elephantiasis lymphatic f...
1589181,1480442,2738,984.250,sofyana girl synthetic aline maxi sydress18845...
2176487,1477669,1574,1600.000,meistar french country style wooden decoration...
1540584,274477,98,744.093,le aventures de telemaques sivies de aventures...
2122598,97645,12743,600.000,bury deeper love passion sordid affair bury de...


In [ ]:
check=df['PRODUCT_LENGTH']

In [ ]:
df['PRODUCT_LENGTH'] = np.log(df['PRODUCT_LENGTH']) #log tranform helps to tranform our data as close to normal skewness. print ('Skewness is', df['PRODUCT_LENGTH'].skew())

In [ ]:
df.corr()

<ipython-input-26-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH
PRODUCT_ID,1.000000,0.255509,0.078707
PRODUCT_TYPE_ID,0.255509,1.000000,0.010789
PRODUCT_LENGTH,0.078707,0.010789,1.000000


In [ ]:
from sklearn.linear_model import Ridge
model = Ridge()

In [ ]:
#df.isnull().any()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df['data'], df['PRODUCT_LENGTH'], test_size=0.3, random_state=3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
cv = CountVectorizer(max_features=2000000)

X_train_vec = cv.fit_transform(df['data'])
# X_test_vec = cv.transform(X_test)


In [ ]:
model.fit(X_train_vec, Y_train)

Ridge()

In [ ]:
from sklearn.metrics import r2_score
pred = model.predict(X_test_vec)

mse = r2_score(Y_test, pred)
mse

0.18051141558281125

In [ ]:
pred=pow(3.14,pred)
pred

array([1629.64690422, 3349.31187522, 2036.10771086, ..., 1237.1545507 ,
       2379.29189873, 3832.23475059])

Testing dataset

In [ ]:
pred.to_csv('predicted_variable.csv', index=False)

AttributeError: ignored

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/test.csv', usecols=['TITLE', 'BULLET_POINTS', 'DESCRIPTION'])

In [ ]:
test_df.head()

,TITLE,BULLET_POINTS,DESCRIPTION
0,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN
1,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...
2,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...
3,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...
4,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN


In [ ]:
test_df = test_df.fillna('')

In [ ]:
test_df.head()

,TITLE,BULLET_POINTS,DESCRIPTION
0,Manuel d'Héliogravure Et de Photogravure En Re...,,
1,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...
2,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...
3,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...
4,"Visual Mathematics, Illustrated by the TI-92 a...",,


In [ ]:
test_df['CONTENT'] = test_df['TITLE'] + ' ' + test_df['BULLET_POINTS'] + ' ' + test_df['DESCRIPTION']

In [ ]:
del test_df['TITLE']
del test_df['BULLET_POINTS']
del test_df['DESCRIPTION']

In [ ]:
test_df.head()

,CONTENT
0,Manuel d'Héliogravure Et de Photogravure En Re...
1,DCGARING Microfiber Throw Blanket Warm Fuzzy P...
2,I-Match Auto Parts Front License Plate Bracket...
3,PinMart Gold Plated Excellence in Service 1 Ye...
4,"Visual Mathematics, Illustrated by the TI-92 a..."


In [ ]:
test_df['CONTENT'] = test_df['CONTENT'].apply(lambda x: remove_html_tags(x))
test_df['CONTENT'] = test_df['CONTENT'].apply(lambda x: remove_punctuations(x))
test_df['CONTENT'] = test_df['CONTENT'].apply(lambda x: remove_stopwords(x))
test_df['CONTENT'] = test_df['CONTENT'].apply(lambda x: lemmatize_text(x))